Redes Neuronales para Lenguaje Natural, 2025

---
# Laboratorio 2

En este laboratorio construiremos un sistema de Question Answering (QA) utilizando el método de Retrieval-Augmented Generation (RAG), que implica el uso de un paso de recuperación de información y un paso de generación de respuesta con LLM.

**Entrega: 18/11**

**Se debe entregar un archivo zip que contenga:**
* Este notebook de Python (.ipynb) completo.
* Los documentos obtenidos y utilizados como fuentes de información según se explica en la parte 1 (opcionalmente se puede entregar un archivo CSV con los textos de cada documento).
* Archivo CSV con el conjunto de preguntas y respuestas como se explica en la parte 5.

**No olvidar mantener todas las salidas de cada región de código en el notebook!**

---



## Parte 0


### Instalación bibliotecas

In [ ]:
if 'google.colab' in str(get_ipython()):
  print('Running on CoLab')
  COLAB = True
else:
  print('Not running on CoLab')
  COLAB = False

#@title Estilo de salida de colab
from IPython.display import HTML, display, clear_output
if COLAB:
    pre_run_cell_fn = lambda: display(HTML('''<style> pre {white-space: pre-wrap;}</style>'''))
    get_ipython().events.register('pre_run_cell', pre_run_cell_fn)

import sys
!{sys.executable} -m pip install transformers
!{sys.executable} -m pip install bitsandbytes
!{sys.executable} -m pip install accelerate
!{sys.executable} -m pip install sentence-transformers
!{sys.executable} -m pip install evaluate
!{sys.executable} -m pip install bert_score
!{sys.executable} -m pip install google-genai
!{sys.executable} -m pip install pymupdf4llm
!{sys.executable} -m pip install langchain-text-splitters
!{sys.executable} -m pip install --upgrade huggingface_hub

clear_output()

### Creación dataset preguntas y respuestas

In [ ]:
import os
TEST_DATASET = "testset.csv"

if not os.path.exists(TEST_DATASET):
    with open(TEST_DATASET, "w", newline="") as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(["question", "answer"])

        # No relacionadas con el dominio
        respuesta_para_pregunta_fuera_de_tema = "No tengo información sobre esa pregunta."
        writer.writerow(["¿Dónde nace el río Uruguay?", respuesta_para_pregunta_fuera_de_tema])
        writer.writerow(["¿En qué año se firmó el tratado de Tordesillas?", respuesta_para_pregunta_fuera_de_tema])
        writer.writerow(["¿Cuál es el presidente actual de Chile?", respuesta_para_pregunta_fuera_de_tema])

        # Necesitan información de más de un chunk
        writer.writerow(["¿La Biblioteca Nacional siempre estuvo en su lugar actual?", "No, la sede de la Biblioteca Nacional ha cambiado varias veces a lo largo de la historia."])

## Parte 1: Procesamiento de los documentos

En esta parte, cada grupo deberá construir y procesar su conjunto de documentos. Esto consiste de los siguientes pasos:

* Elegir un tema dentro de un dominio específico sobre el que trabajar.
* Obtener al menos 5 documentos en español que contengan información sobre el tema elegido.
* Procesar cada documento para extraer el texto del formato original a un string en Python (por ejemplo, extraer el texto de un PDF).

El resultado de esta parte debe ser una lista cargada en memoria que contenga el texto (string) de cada uno de los documentos elegidos.

**Sugerencias:**
* Se recomienda utilizar artículos de wikipedia para simplificar la etapa de extracción del texto (ver la librería [wikipedia-api](https://github.com/martin-majlis/Wikipedia-API/)).
* Opcionalmente puede utilizar documentos PDF, páginas web u otros formatos. En estos casos se sugiere:
  * Utilizar la librería PyPDF2 para procesar documentos PDF.
  * Utilizar la librería LangChain para procesar páginas web, en particular la clase Html2TextTransformer, que convierte HTML a Markdown ([ejemplo de uso](https://python.langchain.com/v0.2/docs/integrations/document_transformers/html2text/)).
* Puede ser conveniente guardar el resultado del procesamiento de los documentos en un archivo CSV (donde cada fila corresponde al texto de un documento) para no tener que repetir este proceso cada vez que se ejecuta el notebook, y en su lugar cargar el archivo CSV.

Usé PyMuPDF en vez de PyPDF2 para poder pasar el pdf a markdown.

Duda -> No sé si usar la página web o el pdf, la página web no tiene todo el documento cargado a priori, hay que scrollear en el iFrame para que aparezca en el html, no sé si suma tanta data tampoco

TO DO -> Revisar como manejar las citas / notas al pie. Enriquecimiento de texto etc

In [ ]:
import pymupdf4llm
import csv

CSV_NAME = "corpus.csv"

def extractTextFromPdf(file_name):
    markdown = pymupdf4llm.to_markdown(file_name)
    return markdown

def writeIntoCsv(file_name, text):
    with open(CSV_NAME, "a", newline="") as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow([file_name, text])

def addFileToCsv(file_name):
    text = extractTextFromPdf(file_name)
    writeIntoCsv(file_name, text)

def createCsv():
    with open(CSV_NAME, "w", newline="") as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(["file_name", "text"])


Consider using the pymupdf_layout package for a greatly improved page layout analysis.


In [ ]:
files = ["Presentacion.pdf", "Acuña de Figueroa.pdf", "Clausura, expolios, intentos de reapertura.pdf", "El edificio de la Biblioteca Nacional.pdf", "La Nueva Biblioteca Nacional.pdf"]

createCsv()
for file_name in files:
  addFileToCsv(file_name)

Los textos resultantes deben estar almacenados en la variable `documents`:

In [ ]:
import pandas as pd
documents = pd.read_csv(CSV_NAME, header = 0, index_col='file_name')

In [ ]:
print(documents)

## Parte 2: Chunking

Una vez que se obtiene el texto de cada documento, se debe realizar la etapa de _chunking_. Esta etapa consiste en dividir cada texto en segmentos más chicos a los que llamamos _chunks_.

Realizar la etapa de _chunking_ de forma automática utilizando un método simple que permita obtener _chunks_ de un largo aproximado de 500 caracteres.

Puede probar con dividir a nivel de caracteres, palabras o incluso párrafos, teniendo en cuenta que el largo de cada _chunk_ no debería exceder demasiado los 500 caracteres.

**Sugerencias:**
* Puede utilizar los splitters disponibles en LangChain ([documentación](https://python.langchain.com/v0.1/docs/modules/data_connection/document_transformers/)) como RecursiveCharacterTextSplitter, aunque no es obligatorio y también es correcto hacer una implementación propia.
* Tener en cuenta que esta etapa es crucial en el resultado final. Cuanto más contextualizados queden los *chunks*, mejor será el rendimiento de la etapa de recuperación de información. Es conveniente minimizar la división de palabras (o párrafos) por la mitad.

In [ ]:
# DOCUMENTO AUXILIAR PARA PRUEBAS
file_path = 'birds.md'

try:
    with open(file_path, 'r', encoding='utf-8') as f:
        birds_document = f.read()
    print(f"--- Successfully loaded content from '{file_path}' ---")

except FileNotFoundError:
    print(f"Error: File not found at '{file_path}'")
    print("Please double-check the filename. It must be an exact match.")
except Exception as e:
    print(f"An error occurred: {e}")

--- Successfully loaded content from 'birds.md' ---


#### Definición de splitters

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_text_splitters import MarkdownHeaderTextSplitter

from enum import StrEnum

class SplitterType(StrEnum):
    RECURSIVE = "Recursive"
    MARKDOWN = "Markdown"

## Recursive Splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 450,
    chunk_overlap  = 40,
    length_function = len,
    is_separator_regex = False,
)

def splitRecursivo(text):
  chunks = text_splitter.split_text(text)
  return chunks

## Markdown Splitter
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
    ("###", "Header 4"),
]

markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on,
    return_each_line=True
)

def splitMarkdown(text):
  markdown_chunks = markdown_splitter.split_text(text)
  chunks = text_splitter.split_documents(markdown_chunks)
  return chunks

/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
def chunk_text(text, splitter=SplitterType.RECURSIVE):

  if splitter == SplitterType.RECURSIVE:
    chunks = splitRecursivo(text)
  elif splitter == SplitterType.MARKDOWN:
    chunks = splitMarkdown(text)

  return chunks # Lista de strings con los chunks del texto

### Codigos para prueba y observar resultados

In [ ]:
## Recursivo

chunks = chunk_text(birds_document,SplitterType.RECURSIVE)

# --- 6. View the Results ---
print(f"\n--- Original Document Length: {len(birds_document)} characters ---")
print(f"--- Total Chunks Generated: {len(chunks)} ---")

print("\n--- Here are the chunks: ---")
for i, chunk in enumerate(chunks[:4]):
    print(f"--- CHUNK {i+1} (Length: {len(chunk)}) ---")
    print(chunk)
    print("-" * 20)


--- Original Document Length: 25998 characters ---
--- Total Chunks Generated: 93 ---

--- Here are the chunks: ---
--- CHUNK 1 (Length: 48) ---
# **A Comprehensive Look at the World of Birds**
--------------------
--- CHUNK 2 (Length: 412) ---
From the iridescent flash of a hummingbird's throat to the imperious stare of a bald eagle, birds represent a vibrant and extraordinarily successful branch of life. They have conquered every continent, from the frozen ice shelves of Antarctica to the densest tropical rainforests and the most arid deserts. With over 10,000 known species, they are the most diverse class of four-limbed vertebrates on the planet.
--------------------
--- CHUNK 3 (Length: 448) ---
What defines a bird? Biologically, they are a group of endothermic (warm-blooded) vertebrates characterized by feathers, toothless beaked jaws, the laying of hard-shelled eggs, a high metabolic rate, a four-chambered heart, and a strong yet lightweight skeleton. Their ability to fly, share

In [ ]:
## Markdown

chunks = chunk_text(birds_document,SplitterType.MARKDOWN)

# --- 6. View the Results ---
print(f"\n--- Original Document Length: {len(birds_document)} characters ---")
print(f"--- Total Chunks Generated: {len(chunks)} ---")

print("\n--- Here are the chunks: ---")
for i, chunk in enumerate(chunks[:4]):
    print(f"--- CHUNK {i+1} (Length: {len(chunk.page_content)}) ---")
    print(chunk)
    print("-" * 20)


--- Original Document Length: 25998 characters ---
--- Total Chunks Generated: 86 ---

--- Here are the chunks: ---
--- CHUNK 1 (Length: 412) ---
page_content='From the iridescent flash of a hummingbird's throat to the imperious stare of a bald eagle, birds represent a vibrant and extraordinarily successful branch of life. They have conquered every continent, from the frozen ice shelves of Antarctica to the densest tropical rainforests and the most arid deserts. With over 10,000 known species, they are the most diverse class of four-limbed vertebrates on the planet.' metadata={'Header 1': '**A Comprehensive Look at the World of Birds**'}
--------------------
--- CHUNK 2 (Length: 448) ---
page_content='What defines a bird? Biologically, they are a group of endothermic (warm-blooded) vertebrates characterized by feathers, toothless beaked jaws, the laying of hard-shelled eggs, a high metabolic rate, a four-chambered heart, and a strong yet lightweight skeleton. Their ability to fly, sha

In [ ]:
## Si se quiere incorporar los títulos al texto

def document_to_list(chunks):
  chunks_list = []

  for chunk in chunks:
      # --- 1. Get and sort headers ---
      header_keys = [k for k in chunk.metadata.keys() if k.startswith('Header ')]

      # Sort the keys numerically, not alphabetically
      # 'key=lambda k: int(k.split(' ')[1])' turns 'Header 10' into 10
      header_keys.sort(key=lambda k: int(k.split(' ')[1]))

      # --- 2. Build the header string ---

      # Get the actual header text for each key
      header_values = [chunk.metadata[k] for k in header_keys]

      # Join them with newlines
      header_string = "\n".join(header_values)

      # --- 3. Concatenate and append ---

      # Add a newline between the headers and the page content
      final_string = f"{header_string}\n{chunk.page_content}"

      chunks_list.append(final_string)

  return chunks_list

chunks_list = document_to_list(chunks)

print(f"--- Generated {len(chunks_list)} strings in chunks_list ---")

print("\n--- Example: First string ---")
print(chunks_list[0])

print("\n--- Example: Fourth string (with two headers) ---")
print(chunks_list[3])

--- Generated 86 strings in chunks_list ---

--- Example: First string ---
**A Comprehensive Look at the World of Birds**
From the iridescent flash of a hummingbird's throat to the imperious stare of a bald eagle, birds represent a vibrant and extraordinarily successful branch of life. They have conquered every continent, from the frozen ice shelves of Antarctica to the densest tropical rainforests and the most arid deserts. With over 10,000 known species, they are the most diverse class of four-limbed vertebrates on the planet.

--- Example: Fourth string (with two headers) ---
**A Comprehensive Look at the World of Birds**
**The Avian Lineage: From Dinosaurs to Modern Birds**
The story of birds begins not in the sky, but on the ground, among the bipedal, meat-eating dinosaurs of the Jurassic period. For a long time, the link was controversial, but the fossil evidence is now overwhelming. Birds are not just *related* to dinosaurs; they *are* dinosaurs. Specifically, they are the last 

#### Métricas

Ajustar a los chunks de los textos reales que usemos

In [ ]:
all_lengths = [len(chunk.page_content) for chunk in chunks]

ordered_lengths = sorted(all_lengths)

average_length = sum(all_lengths) / len(all_lengths)

print(f"Average chunk length: {average_length:.2f} characters")
print(f"Median chunk length: {ordered_lengths[42]} characters")

Average chunk length: 294.42 characters
Median chunk length: 296 characters


## Experimentando alternativas

Esta parte tiene ideas que podrian ser utiles para la recuperación. Por ahora es solo copy-paste de código portencialmente útil

In [ ]:
import pprint

def build_header_index(chunks):
    """Builds a nested dictionary index from chunk metadata."""
    header_index = {}

    for chunk in chunks:
        # Get all header keys and sort them numerically
        # (e.g., 'Header 1', 'Header 2', 'Header 10')
        header_keys = sorted(
            [k for k in chunk.metadata.keys() if k.startswith('Header ')],
            key=lambda k: int(k.split(' ')[1])
        )

        # Get the actual header text values
        header_path = [chunk.metadata[k] for k in header_keys]

        # --- Build the nested dictionary ---
        current_level = header_index
        for header in header_path:
            if header not in current_level:
                current_level[header] = {}  # Create a new branch
            current_level = current_level[header] # Move down the tree

    return header_index

# --- Run the function and print the result ---
index = build_header_index(chunks)

print("--- Nested Header Index ---")
pprint.pprint(index)

--- Nested Header Index ---
{'**A Comprehensive Look at the World of Birds**': {'**A Shared World: Birds and Humans**': {'**Symbols, Messengers, and Myths**': {},
                                                                                             '**The Joy of Birding**': {},
                                                                                             '**Threats and Conservation**': {}},
                                                    '**A World of Variety: A Tour of Major Bird Orders**': {},
                                                    '**From Egg to Adult: Reproduction and Life Cycle**': {'**Altricial vs. Precocial: Raising the Young**': {},
                                                                                                           '**Building a Home: Nests and Incubation**': {},
                                                                                                           '**Finding a Mate: Courtship Rituals**': {}},
   

In [ ]:
def get_chunks_by_path(chunks, header_path):
    """
    Finds all chunks that match a specific header path.

    A chunk matches if its metadata path starts with the provided header_path.
    """
    matching_chunks = []
    len_path = len(header_path)

    for chunk in chunks:
        # Get all header keys and sort them numerically
        header_keys = sorted(
            [k for k in chunk.metadata.keys() if k.startswith('Header ')],
            key=lambda k: int(k.split(' ')[1])
        )

        # Get the chunk's full header path
        chunk_path = [chunk.metadata[k] for k in header_keys]

        # Check if the chunk's path starts with the user's path
        if chunk_path[:len_path] == header_path:
            matching_chunks.append(chunk)

    return matching_chunks

In [ ]:
path_h2 = [
    '**A Comprehensive Look at the World of Birds**',
    '**The Avian Lineage: From Dinosaurs to Modern Birds**'
]

results_h2 = get_chunks_by_path(chunks, path_h2)

print(f"\n--- Found {len(results_h2)} chunks for path: {path_h2} ---")
for i, chunk in enumerate(results_h2):
    print(f"Chunk {i+1} Content: {chunk.page_content}")
    print(f"Chunk {i+1} Metadata: {chunk.metadata}\n")


--- Found 8 chunks for path: ['**A Comprehensive Look at the World of Birds**', '**The Avian Lineage: From Dinosaurs to Modern Birds**'] ---
Chunk 1 Content: The story of birds begins not in the sky, but on the ground, among the bipedal, meat-eating dinosaurs of the Jurassic period. For a long time, the link was controversial, but the fossil evidence is now overwhelming. Birds are not just *related* to dinosaurs; they *are* dinosaurs. Specifically, they are the last surviving lineage of theropod dinosaurs, the same group that included *Velociraptor* and *Tyrannosaurus rex*.
Chunk 1 Metadata: {'Header 1': '**A Comprehensive Look at the World of Birds**', 'Header 2': '**The Avian Lineage: From Dinosaurs to Modern Birds**'}

Chunk 2 Content: The most famous transitional fossil is *Archaeopteryx*, a crow-sized creature that lived about 150 million years ago. Discovered in Germany, its fossils show a mosaic of features. Like a reptile, it had a full set of teeth, a flat breastbone, a long 

In [ ]:
chunks = []
for document in documents:
  chunks += chunk_text(document)

## Parte 3: Recuperación de información

En esta parte vamos a implementar el método de recuperación de información que nos permitirá obtener los _chunks_ más relevantes para la pregunta.

En primer lugar, cargamos el modelo Bi-Encoder que utilizaremos para generar los embeddings utilizando la librería sentence_transformers.

Se utiliza el modelo multilingüe [intfloat/multilingual-e5-large](https://huggingface.co/intfloat/multilingual-e5-large), fine-tuning del modelo `xlm-roberta-large` para la tarea de generación de sentence embeddings.

Se pueden explorar otros modelos Bi-Encoder, e incluso modelos Cross-Encoder o del tipo ColBERT. En HuggingFace se puede consultar el siguiente [leaderboard](https://huggingface.co/spaces/mteb/leaderboard) que compara varios modelos de este tipo en diferentes tareas.

In [ ]:
from sentence_transformers import SentenceTransformer

model_emb = SentenceTransformer("intfloat/multilingual-e5-large")

A continuación se debe generar las representaciones vectoriales para todos los _chunks_ ([ejemplo de uso](https://huggingface.co/intfloat/multilingual-e5-large#support-for-sentence-transformers)).

**Observación:** El modelo que estamos usando espera que los _chunks_ comiencen con el prefijo `passage: ` por lo que será necesario agregarlo al inicio de todos los _chunks_.

In [ ]:
def prepare_chunks_for_embedding(chunks):
    """
    Prepara los chunks agregando el prefijo requerido por el modelo.
    Maneja tanto strings como objetos Document de LangChain.
    """
    prepared_chunks = []
    for chunk in chunks:
        # Si es un objeto Document (del markdown splitter), extraer el contenido
        if hasattr(chunk, 'page_content'):
            text = chunk.page_content
        else:
            # Si es un string simple
            text = chunk

        prepared_chunks.append(f"passage: {text}")

    return prepared_chunks

chunks_with_prefix = prepare_chunks_for_embedding(chunks)

print(f"Generando embeddings para {len(chunks_with_prefix)} chunks...")
chunk_embeddings = model_emb.encode(chunks_with_prefix, normalize_embeddings=True)

print(f"Embeddings generados. Dimensión: {chunk_embeddings.shape}")

Generando embeddings para 1 chunks...
Embeddings generados. Dimensión: (1, 1024)


Por último, se debe implementar el algoritmo de búsqueda de los embeddings más cercanos para un embedding dado.

**Sugerencias:**
* Utilizar la clase NearestNeighbors de sklearn ([documentación](https://scikit-learn.org/dev/modules/generated/sklearn.neighbors.NearestNeighbors.html#sklearn.neighbors.NearestNeighbors)).

In [ ]:
from sklearn.neighbors import NearestNeighbors

nn_model = NearestNeighbors(
    n_neighbors=5,  # Por ahora buscamos 5 vecinos, ajustamos despues
    metric='cosine',
    algorithm='brute'  # 'brute' supuestamente es más preciso para datasets chicos/medianos
)

nn_model.fit(chunk_embeddings)

print("Modelo de vecinos más cercanos entrenado")

Modelo de vecinos más cercanos entrenado


In [ ]:
def retrieve_chunks(query, top_k=3):
    """
    Returns:
        Una tupla (chunks_recuperados, distancias, indices)
        - chunks_recuperados: Lista de textos de los chunks más relevantes
        - distancias: Distancias coseno a cada chunk
        - indices: Índices de los chunks en la lista original
    """
    query_with_prefix = f"query: {query}"

    query_embedding = model_emb.encode([query_with_prefix], normalize_embeddings=True)

    distances, indices = nn_model.kneighbors(query_embedding, n_neighbors=top_k)

    # Extraer los chunks correspondientes
    retrieved_chunks = []
    for idx in indices[0]:
        chunk = chunks[idx]
        # Manejar tanto strings como objetos Document
        if hasattr(chunk, 'page_content'):
            retrieved_chunks.append(chunk.page_content)
        else:
            retrieved_chunks.append(chunk)

    return retrieved_chunks, distances[0], indices[0]

## Parte 4: Generación de respuestas

### Configuración de LLM

En esta parte, implementaremos un wrapper flexible que permite experimentar con diferentes modelos de lenguaje:

1. **Llama 3.1** (modelo abierto): Utilizaremos el modelo Meta-Llama-3.1-8B-Instruct a través de HuggingFace.
2. **Gemini 2.0 Flash** (modelo cerrado): Utilizaremos la API de Google Gemini.

Para **Llama 3.1**, es necesario:
- Crearse una cuenta de HuggingFace (https://huggingface.co/)
- Aceptar los términos para usar el modelo en HuggingFace: https://huggingface.co/meta-llama/Meta-Llama-3.1-8B-Instruct
- Crear un token de HuggingFace con permiso de lectura: https://huggingface.co/settings/tokens

Para **Gemini**, es necesario:
- Obtener una API key de Google AI Studio: https://aistudio.google.com/app/apikey

In [ ]:
# Ejecutar para conectarse a HuggingFace (solo necesario para Llama 3.1)
from huggingface_hub import notebook_login

if COLAB:
  notebook_login()
else:
    # Loguear localmente

ImportError: The `notebook_login` function can only be used in a notebook (Jupyter or Colab) and you need the `ipywidgets` module: `pip install ipywidgets`.

### Definición del Wrapper de LLM

A continuación se define una clase abstracta que permite intercambiar fácilmente entre diferentes modelos de lenguaje.

In [ ]:
from abc import ABC, abstractmethod
from typing import Optional
import os
from getpass import getpass

class LLMWrapper(ABC):
    """Clase base abstracta para wrappers de modelos de lenguaje."""

    @abstractmethod
    def generate(self, prompt: str, temperature: float = 0.0, max_tokens: int = 500) -> str:
        pass

    @abstractmethod
    def get_model_name(self) -> str:
        pass


class LlamaWrapper(LLMWrapper):
    """Wrapper para el modelo Llama 3.1 de Meta vía HuggingFace."""

    def __init__(self):
        from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
        import torch

        print("Inicializando Llama 3.1...")

        self.tokenizer = AutoTokenizer.from_pretrained(
            "meta-llama/Meta-Llama-3.1-8B-Instruct")

        # Configuración de cuantización a 4 bits (para mejorar eficiencia)
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.bfloat16)

        # Inicializar el modelo
        self.model = AutoModelForCausalLM.from_pretrained(
            "meta-llama/Meta-Llama-3.1-8B-Instruct",
            quantization_config=bnb_config,
            device_map="auto")

        print("Llama 3.1 inicializado correctamente")

    def generate(self, prompt: str, temperature: float = 0.0, max_tokens: int = 500) -> str:
        from transformers import GenerationConfig, pipeline

        # Configuración de temperatura
        generation_config = GenerationConfig(
            temperature=temperature if temperature > 0 else None,
            do_sample=temperature > 0)

        # Inicializar pipeline para generación de texto
        pipe = pipeline(
            "text-generation",
            model=self.model,
            config=generation_config,
            tokenizer=self.tokenizer,
            pad_token_id=self.tokenizer.eos_token_id)

        # Generar texto
        output = pipe(
            prompt,
            return_full_text=False,
            max_new_tokens=max_tokens)

        return output[0]['generated_text']

    def get_model_name(self) -> str:
        return "Llama-3.1-8B-Instruct"


class GeminiWrapper(LLMWrapper):
    """Wrapper para el modelo Gemini 2.0 Flash de Google."""

    def __init__(self, api_key: Optional[str] = None):
        from google import genai

        print("Inicializando Gemini 2.0 Flash...")

        # Obtener API key
        if api_key is None:
            api_key = os.environ.get("GEMINI_API_KEY")

        if api_key is None:
            print("No se encontró la API key de Gemini en las variables de entorno.")
            api_key = getpass("Por favor, ingrese su API key de Gemini: ")
            # Guardar en variables de entorno para esta sesión
            os.environ["GEMINI_API_KEY"] = api_key

        self.client = genai.Client(api_key=api_key)

        print("Gemini 2.0 Flash inicializado correctamente")

    def generate(self, prompt: str, temperature: float = 0.0, max_tokens: int = 500) -> str:
        from google.genai import types

        config = types.GenerateContentConfig(
            temperature=temperature if temperature > 0 else 0.0,
            max_output_tokens=max_tokens)

        # Generar respuesta
        response = self.client.models.generate_content(
            model="gemini-2.0-flash-exp",
            contents=prompt,
            config=config)

        return response.text

    def get_model_name(self) -> str:
        return "Gemini-2.0-Flash"

### Instanciar modelos

Seleccionar qué modelo(s) se va a inicializar. Se puede inicializar ambos para facilitar la experimentación posterior.

In [ ]:
# Descomentar el(los) modelo(s) que se quiera utilizar

llama_model = LlamaWrapper()

gemini_model = GeminiWrapper()

# Seleccionar el modelo activo para los experimentos
active_model = llama_model  # o gemini_model

print(f"\nModelo activo: {active_model.get_model_name()}")

### Función auxiliar para generación de respuestas

Esta función utiliza el modelo activo seleccionado anteriormente.

In [ ]:
def get_response(
    prompt: str,
    model: LLMWrapper = None,
    temp: float = 0.0,
    max_tok: int = 500
) -> str:
    if model is None:
        model = active_model

    return model.generate(prompt, temperature=temp, max_tokens=max_tok)

### Crear prompt y generar respuesta

Escribir la función `create_prompt(question, use_chat_template=True)` que dada una pregunta, genere la prompt que se utilizará para generar la respuesta. Tener en cuenta que se debe realizar la búsqueda semántica de los _chunks_ más cercanos a la pregunta utilizando lo implementado en la parte 3.

**Observación:** Al igual que para los _chunks_, el modelo Bi-Encoder espera que la pregunta comience con un prefijo especial: `query: ` por lo que será necesario agregarlo al inicio de la pregunta para generar el embedding.

**Sugerencias:**
* Puede probar con distintas cantidades de _chunks_ recuperados, pero se sugiere comenzar con 3. Tener en cuenta que más _chunks_ recuperados y agregados en la prompt implica mayor uso de memoria en inferencia.
* El parámetro `use_chat_template` permite controlar si se aplica el template de chat de Llama (necesario para Llama 3.1, opcional para Gemini). Para Llama usar `True`, para Gemini se puede probar con `True` o `False` según el formato que se prefiera.

In [ ]:
def create_prompt(question, use_chat_template=True, model_for_template=None, top_k=3):
    """
    Crea el prompt para el modelo de lenguaje incluyendo contexto recuperado.

    Args:
        question: La pregunta del usuario
        use_chat_template: Si True, aplica el template de chat de Llama
        model_for_template: Modelo del cual usar el tokenizer (solo para Llama)
        top_k: Número de chunks a recuperar (por defecto 3)

    Returns:
        El prompt formateado
    """
    # 1. Recuperar chunks relevantes usando búsqueda semántica
    # La función retrieve_chunks ya maneja el prefijo "query:" internamente
    retrieved_chunks, _, _ = retrieve_chunks(question, top_k=top_k)

    # 2. Construir el contexto con los chunks recuperados
    context = "\n\n".join(retrieved_chunks)

    # 3. Crear el mensaje del sistema y del usuario
    system_message = """Eres un asistente experto en responder preguntas basándote únicamente en el contexto proporcionado.

Instrucciones:
- Si la información para responder la pregunta está en el contexto, proporciona una respuesta clara y concisa.
- Si la información NO está en el contexto, responde: "Lo siento, no cuento con información para responder esa pregunta."
- No inventes información que no esté en el contexto.
- Responde en español."""

    user_message = f"""Contexto:
{context}

Pregunta: {question}"""

    # 4. Aplicar chat template si corresponde
    if use_chat_template:
        # Usar tokenizer de Llama para aplicar template
        if model_for_template is None and isinstance(active_model, LlamaWrapper):
            model_for_template = active_model

        if isinstance(model_for_template, LlamaWrapper):
            messages = [
                {"role": "system", "content": system_message},
                {"role": "user", "content": user_message}
            ]
            prompt = model_for_template.tokenizer.apply_chat_template(
                messages, tokenize=False, add_generation_prompt=True
            )
        else:
            # Para Gemini u otros modelos sin template específico
            prompt = f"{system_message}\n\n{user_message}"
    else:
        # Sin template, formato simple
        prompt = f"{system_message}\n\n{user_message}"

    return prompt

Probar la prompt anterior con un ejemplo.

In [ ]:
question = ""  # Completar con una pregunta adecuada al contexto

# Crear prompt según el modelo activo
use_template = isinstance(active_model, LlamaWrapper)  # True para Llama, False para Gemini
prompt = create_prompt(question, use_chat_template=use_template)

print(f"MODELO: {active_model.get_model_name()}")
print("\nPROMPT:")
print(prompt)
print("\nRESPUESTA:")
print(get_response(prompt))

## Parte 5: Evaluación
A continuación vamos a evaluar la solución construida. Para ello, se deben seguir los siguientes pasos:

* Construir un conjunto de evaluación de forma manual que contenga al menos 12 preguntas y respuestas con las siguientes características:
  * Al menos 3 preguntas deben necesitar información presente en más de un _chunk_ para ser respondidas correctamente.
  * Al menos 3 preguntas no deben estar relacionadas con el dominio, y su respuesta de referencia debe ser algo similar a: "Lo siento, no cuento con información para responder esa pregunta."
* El conjunto debe estar en un archivo CSV llamado testset.csv, con las columnas "question" y "answer".

Se deberá realizar al menos tres experimentos diferentes y evaluar sobre el mismo conjunto de test con la métrica BERTScore. Los experimentos deben variar en al menos uno de los siguientes elementos:
* Método de chunking
* Modelo (o método) de retrieval
* Modelo de generación (LLM)
* Método de prompting (se puede probar con few-shot, chain of thought, etc)
* Otros aspectos que considere relevantes a probar

A continuación se definen funciones auxiliares para la evaluación.


In [ ]:
import evaluate
import numpy as np
from tqdm.notebook import tqdm

def generate_predictions(questions, model=None, use_chat_template=True):
    if model is None:
        model = active_model

    predictions = []
    for question in tqdm(questions, desc=f"Generando con {model.get_model_name()}"):
        prompt = create_prompt(question, use_chat_template=use_chat_template)
        prediction = get_response(prompt, model=model)
        predictions.append(prediction)

    return predictions

def evaluate_predictions(predictions, references, experiment_name=""):
    """Evalúa predicciones usando BERTScore."""
    bertscore = evaluate.load("bertscore")
    results = bertscore.compute(predictions=predictions, references=references, lang='es')

    metrics = {
        'precision': np.array(results['precision']).mean(),
        'recall': np.array(results['recall']).mean(),
        'f1': np.array(results['f1']).mean()
    }

    if experiment_name:
        print(f"\n=== Resultados: {experiment_name} ===")
    print(f"BERTScore P:  {metrics['precision']:.3f}")
    print(f"BERTScore R:  {metrics['recall']:.3f}")
    print(f"BERTScore F1: {metrics['f1']:.3f}")

    return metrics

In [ ]:
import pandas as pd

# Leer el conjunto de evaluación
df = pd.read_csv("testset.csv")

# Obtener preguntas y respuestas
questions = df["question"].tolist()
references = df["answer"].tolist()

Evalúe los experimentos realizados.

In [ ]:
# Evaluar experimentos
# Almacenar resultados para comparación posterior
results_dict = {}

# ============================================================================
# EXPERIMENTO 1: Llama 3.1 con chat template
# ============================================================================
exp1_model = llama_model
exp1_name = "Exp1: Llama 3.1 con chat template"

predictions_exp1 = generate_predictions(questions, model=exp1_model, use_chat_template=True)
results_dict[exp1_name] = evaluate_predictions(predictions_exp1, references, exp1_name)


# ============================================================================
# EXPERIMENTO 2: Gemini 2.0 Flash
# ============================================================================
# exp2_model = gemini_model
# exp2_name = "Exp2: Gemini 2.0 Flash"
#
# predictions_exp2 = generate_predictions(questions, model=exp2_model, use_chat_template=False)
# results_dict[exp2_name] = evaluate_predictions(predictions_exp2, references, exp2_name)


# ============================================================================
# EXPERIMENTO 3: Otro experimento (acá variamos lo que dijimos)
# ============================================================================
# exp3_model = active_model  # Usar el modelo que ande mejor, por ej.
# exp3_name = "Exp3: [Descripción del experimento]"
#
# # Ejemplo: se puede modificar create_prompt para usar diferente estrategia
# predictions_exp3 = generate_predictions(questions, model=exp3_model, use_chat_template=True)
# results_dict[exp3_name] = evaluate_predictions(predictions_exp3, references, exp3_name)


# ============================================================================
# RESUMEN DE RESULTADOS
# ============================================================================
print("\n" + "="*80)
print("RESUMEN DE TODOS LOS EXPERIMENTOS")
print("="*80)

import pandas as pd
summary_df = pd.DataFrame(results_dict).T
summary_df.columns = ['Precision', 'Recall', 'F1']
print(summary_df.to_string())

### Reporte de resultados

Reportar los resultados obtenidos en los experimentos realizados completando la siguiente tabla:

| Exp | Descripción | P BERTScore | R BERTScore | F BERTScore |
|-----|-------------|-------------|-------------|-------------|
| 1 | | | | |
| 2 | | | | |
| 3 | | | | |

Responda las siguientes preguntas:

1. Explique brevemente las diferencias en los experimentos realizados, ¿Qué aspectos se varió en el pipeline de RAG?

2. ¿Son consistentes los resultados obtenidos con lo que esperaba?

3. ¿Le parece que la métrica BERTScore está capturando correctamente las diferencias de los distintos experimentos realizados?

(sus respuestas aquí)
